In [ ]:
import cvxpy as cp
import itertools
import numpy as np
from numpy import unravel_index
import copy

In [ ]:
def auxiliary_constraints(template):
    """
    Generates auxiliary constraint matrices based on a given Sudoku template.

    Args:
    template: A 9x9 numpy array representing the Sudoku puzzle. Cells with numbers 1-9 are pre-filled,
              while empty cells contain zeros.

    Returns:
    lower_constraints: A list of 9 matrices, each with the constraints that force the indicator variables
                       for a given number (1-9) to be at least 1 in the positions where that number appears
                       in the template. All other positions are zero.
    upper_constraints: A list of 9 matrices filled with ones, indicating the upper bound constraints for
                       all indicator variables.
    """
    lower_constraints = [np.array([int(template[i][j] == number) for i in range(9) for j in range(9)]).reshape((9, 9))
                         for number in range(1, 10)]
    upper_constraints = [np.ones((9, 9)) for _ in range(9)]

    return lower_constraints, upper_constraints

In [ ]:
def generate_constraints(puzzle, variables):
    """
    Generates constraints for the Sudoku puzzle using cvxpy variables. The constraints ensure that
    the solution adheres to the rules of Sudoku, which are:
      1. Each number (from 1 to 9) must appear exactly once in each row.
      2. Each number must appear exactly once in each column.
      3. Each number must appear exactly once in each 3x3 sub-grid.
      4. The known numbers from the puzzle must remain fixed in their positions.

    Args:
    puzzle: A 9x9 array-like object with integers from 0 to 9, where 0 represents an empty cell.
    variables: A list of 9 cvxpy Variables, where each variable represents the presence (1)
               or absence (0) of numbers 1 through 9 in the 9x9 Sudoku grid.

    Returns:
    constraints: A list of cvxpy constraints that incorporate the Sudoku rules.
    """
    lc, uc = auxiliary_constraints(puzzle)
    constraints = []

    #constraints ensuring that lower and upper boundaries hold
    for layer in range(9):
      for row in range(9):
        for col in range(9):
          constraints.append(variables[layer][row][col] >= lc[layer][row][col])
          constraints.append(variables[layer][row][col] <= uc[row][col])

    ## constraints 1 and 2
    for layer in range(9):
        for axis in range(9):
            constraints.append(cp.sum(variables[layer][axis, :]) == 1)  # 1
            constraints.append(cp.sum(variables[layer][:, axis]) == 1)  # 2

    ## constraints for submatrices
    for layer in range(9):
        for submatrix_row in range(3):
            for submatrix_col in range(3):
                constraints.append(cp.sum(variables[layer][submatrix_row*3:(submatrix_row+1)*3, submatrix_col*3:(submatrix_col+1)*3]) == 1)

    ## constraints that make 1 cell contain 1 number
    for i in range(9):
        for j in range(9):
            constraints.append(cp.sum([variables[layer][i, j] for layer in range(9)]) == 1)

    ## constraints ensuring that existing numbers stay in their places
    for i in range(9):
        for j in range(9):
            if puzzle[i][j] != 0:
                num = puzzle[i][j] - 1
                for k in range(9):
                    if k == num:
                        constraints.append(variables[k][i, j] == 1)
                    else:
                        constraints.append(variables[k][i, j] == 0)

    return constraints

In [ ]:
def solve_sudoku_auxiliary(puzzle):
    """
    An auxiliary function to attempt to solve the Sudoku puzzle using linear programming.

    Args:
    puzzle: A 9x9 numpy array representing the Sudoku puzzle with zeroes indicating empty cells.
    const: A list of additional constraints (each an array-like of format [row, column, value])
           that must be applied to the puzzle, often coming from the branch and bound process.

    Returns:
    answer: A 9x9 numpy array representing the partially/completely solved puzzle. If completely solved, all cells contain integers from 1 to 9.
    variables: a copy of the cvxpy solution
    flag: A boolean indicating whether the puzzle was solved to an integer solution (True) or if branching is required (False).
    """
    variables = [cp.Variable((9, 9)) for _ in range(9)]
    objective = cp.Minimize(cp.sum(cp.maximum(*[variables[i] for i in range(9)])))
    constraints = generate_constraints(puzzle, variables) #генерируем констрейнты
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    status = prob.status #получаем статус задачи -- нам нужно, чтобы там было 'optimal'

    if status == 'infeasible': #нет смысла идти дальше, если так
      return [], [], prob.status

    solution = np.zeros((9, 9), dtype=int) #неинтересно -- заполняем полное или частичное решение по итогам линейного программирования
    for layer in range(9):
        for i in range(9):
            for j in range(9):
                if variables[layer].value[i, j] >= 0.995: # по условию -- устраняем аппроксимации
                    solution[i, j] = layer + 1

    variables_copy = [np.round(variables[i].value, 3) for i in range(len(variables))] #copy variables
    return solution, variables_copy, prob.status

In [ ]:
def check(sol): #чекер на infeasibility и на наличие нулей в решении
  if len(sol) == 0:
    return False
  for row in range(9):
    for col in range(9):
      if sol[row][col] == 0:
        return False
  return True

def find_cell_to_branch(matrix): #найти координаты клеток, где нецелые значения
    fractional_parts, _ = np.modf(matrix) #бьём numpy.float64() на целую и дробную часть
    non_integer_indices = np.where(fractional_parts != 0) #находим те места, где есть нецелые числа

    if len(non_integer_indices) != 0:
      non_integer_coords = list(zip(non_integer_indices[0], non_integer_indices[1])) #упаковываем
    else: non_integer_coords = []

    return non_integer_coords

def match_cells_to_branch(variables): #находим пару для branch and bound. благодаря objective функции, для каждой клетки находится ровно две пары
  cells = []
  for layer in range(len(variables)):
    if len(find_cell_to_branch(variables[layer]))!=0:
      cells.append([layer, find_cell_to_branch(variables[layer])]) #пары слой - координаты
  coords, layers = cells[-1][1][0], [cells[-1][0]] #берём последнюю. так удобнее. (лень было делать хипдикт (это же не принципиально, да? (решает же???????)))

  for cell in cells: #находим пары предыдущим слою и клетке
    if coords in cell[1] and cell[0] != layers[0]:
      layers.append(cell[0])
  print(coords, layers) #for convenience
  return coords, layers

In [ ]:
def branch_and_bound(puzzle, cells, states_prev):
  states = []

  for i in range(len(cells[1])):
      state = copy.deepcopy(puzzle)
      state[cells[0][0]][cells[0][1]] = cells[1][i]+1
      states.append(state)
  for prev_state in states_prev:
    states.append(prev_state)
  return states

In [ ]:
def solve_sudoku(puzzle):
    """
    Solves the Sudoku puzzle using a branch and bound algorithm. This function serves as the main driver that
    iteratively processes subproblems until a complete solution is found or all options are exhausted.

    Args:
    puzzle: A 9x9 list of lists representing the Sudoku grid where a 0 value indicates a cell to be solved.

    Returns:
    A tuple (solved_puzzle, constraints) where:
    - solved_puzzle: A fully solved 9x9 Sudoku grid.
    - constraints: A list of constraints that lead to the solution, represented as tuples (see assignment for details).

    The function uses a priority queue to manage branching, with lower objective values given priority. The
    branch and bound process continues until a solution with all integer values is found for the puzzle.
    """

    solution, variables, status = solve_sudoku_auxiliary(puzzle)
    states = []
    while True: #branch and bound cycle

        if check(solution) == True: #if the problem is considered solved, break
          break
        if status != 'infeasible': #all bnb steps either lead to infeasible problem status or towards an optimal status. this cycle is for 'optimal'
          cells_to_branch = match_cells_to_branch(variables) #find coordinates of an ambiguous cell and its possible states
          states = branch_and_bound(solution, cells_to_branch, states) #get puzzle states for possible cell states
          solution = states[0]
          states.pop(0) #try the first possible state
          solution, variables, status = solve_sudoku_auxiliary(solution)
        if status == 'infeasible': #if the first possible fails, we try the other one.
          solution = states[0]
          states.pop(0)
          solution, variables, status = solve_sudoku_auxiliary(solution)

    return solution

In [ ]:
def test():
    """
    This is a sample main function that shows examples and expected
    outputs of calling the simplex and solve_sudoku functions.
    """

    sudoku_puzzle = [[8, 5, 9, 6, 1, 2, 4, 3, 7],
            [7, 2, 0, 8, 5, 4, 1, 6, 9],
            [1, 6, 4, 3, 7, 9, 5, 2, 8],
            [9, 8, 6, 1, 4, 7, 3, 5, 2],
            [3, 7, 5, 2, 6, 8, 9, 1, 4],
            [2, 4, 1, 5, 9, 0, 7, 8, 6],
            [4, 3, 2, 9, 8, 1, 6, 7, 5],
            [6, 1, 7, 0, 2, 5, 8, 9, 3],
            [5, 9, 8, 7, 3, 6, 2, 4, 1]]

    sudoku_sol = [[8, 5, 9, 6, 1, 2, 4, 3, 7],
            [7, 2, 3, 8, 5, 4, 1, 6, 9],
            [1, 6, 4, 3, 7, 9, 5, 2, 8],
            [9, 8, 6, 1, 4, 7, 3, 5, 2],
            [3, 7, 5, 2, 6, 8, 9, 1, 4],
            [2, 4, 1, 5, 9, 3, 7, 8, 6],
            [4, 3, 2, 9, 8, 1, 6, 7, 5],
            [6, 1, 7, 4, 2, 5, 8, 9, 3],
            [5, 9, 8, 7, 3, 6, 2, 4, 1]]

    print('=== solve_sudoku')
    print(' + Expected value:\n', np.array(sudoku_sol), '\n')
    print(' +     Your value:\n {}'.format(solve_sudoku(sudoku_puzzle)), '\n', '\n', solve_sudoku(sudoku_puzzle) == sudoku_sol, '\n')

    sudoku_puzzle = [[8, 5, 0, 0, 0, 2, 4, 0, 0],
            [7, 2, 0, 0, 0, 0, 0, 0, 9],
            [0, 0, 4, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 7, 0, 0, 2],
            [3, 0, 5, 0, 0, 0, 9, 0, 0],
            [0, 4, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 8, 0, 0, 7, 0],
            [0, 1, 7, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 3, 6, 0, 4, 0]]

    sudoku_sol = [[8, 5, 9, 6, 1, 2, 4, 3, 7],
            [7, 2, 3, 8, 5, 4, 1, 6, 9],
            [1, 6, 4, 3, 7, 9, 5, 2, 8],
            [9, 8, 6, 1, 4, 7, 3, 5, 2],
            [3, 7, 5, 2, 6, 8, 9, 1, 4],
            [2, 4, 1, 5, 9, 3, 7, 8, 6],
            [4, 3, 2, 9, 8, 1, 6, 7, 5],
            [6, 1, 7, 4, 2, 5, 8, 9, 3],
            [5, 9, 8, 7, 3, 6, 2, 4, 1]]

    print('=== solve_sudoku')
    print(' + Expected value:\n', np.array(sudoku_sol), '\n')
    print(' +     Your value:\n {}'.format(solve_sudoku(sudoku_puzzle)), '\n', '\n', solve_sudoku(sudoku_puzzle) == sudoku_sol, '\n')

    sudoku_puzzle = [[4, 8, 0, 3, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 7, 1],
            [0, 2, 0, 0, 0, 0, 0, 0, 0],
            [7, 0, 5, 0, 0, 0, 0, 6, 0],
            [0, 0, 0, 2, 0, 0, 8, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 7, 6, 0, 0, 0],
            [3, 0, 0, 0, 0, 0, 4, 0, 0],
            [0, 0, 0, 0, 5, 0, 0, 0, 0]]

    sudoku_sol = [[4, 8, 7, 3, 1, 2, 6, 9, 5],
            [5, 9, 3, 6, 8, 4, 2, 7, 1],
            [1, 2, 6, 5, 9, 7, 3, 8, 4],
            [7, 3, 5, 8, 4, 9, 1, 6, 2],
            [9, 1, 4, 2, 6, 5, 8, 3, 7],
            [2, 6, 8, 7, 3, 1, 5, 4, 9],
            [8, 5, 1, 4, 7, 6, 9, 2, 3],
            [3, 7, 9, 1, 2, 8, 4, 5, 6],
            [6, 4, 2, 9, 5, 3, 7, 1, 8]]

    print('=== solve_sudoku')
    print(' + Expected value: \n', np.array(sudoku_sol), '\n')
    print(' +     Your value: \n {}'.format(solve_sudoku(sudoku_puzzle)), '\n', '\n',solve_sudoku(sudoku_puzzle) == sudoku_sol)

test()

=== solve_sudoku
 + Expected value:
 [[8 5 9 6 1 2 4 3 7]
 [7 2 3 8 5 4 1 6 9]
 [1 6 4 3 7 9 5 2 8]
 [9 8 6 1 4 7 3 5 2]
 [3 7 5 2 6 8 9 1 4]
 [2 4 1 5 9 3 7 8 6]
 [4 3 2 9 8 1 6 7 5]
 [6 1 7 4 2 5 8 9 3]
 [5 9 8 7 3 6 2 4 1]] 

 +     Your value:
 [[8 5 9 6 1 2 4 3 7]
 [7 2 3 8 5 4 1 6 9]
 [1 6 4 3 7 9 5 2 8]
 [9 8 6 1 4 7 3 5 2]
 [3 7 5 2 6 8 9 1 4]
 [2 4 1 5 9 3 7 8 6]
 [4 3 2 9 8 1 6 7 5]
 [6 1 7 4 2 5 8 9 3]
 [5 9 8 7 3 6 2 4 1]] 
 
 [[ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]] 

=== solve_sudoku
 + Expected value:
 [[8 

In [ ]:
sudoku_ultimate =[
    [0,0,0,8,0,1,0,0,0],
    [0,0,0,0,0,0,0,4,3],
    [5,0,0,0,0,0,0,0,0],
    [0,0,0,0,7,0,8,0,0],
    [0,0,0,0,0,0,1,0,0],
    [0,2,0,0,3,0,0,0,0],
    [6,0,0,0,0,0,0,7,5],
    [0,0,3,4,0,0,0,0,0],
    [0,0,0,2,0,0,6,0,0],
]
sudoku_ultimate_sol =[
       [2, 3, 7, 8, 4, 1, 5, 6, 9],
       [1, 8, 6, 7, 9, 5, 2, 4, 3],
       [5, 9, 4, 3, 2, 6, 7, 1, 8],
       [3, 1, 5, 6, 7, 4, 8, 9, 2],
       [4, 6, 9, 5, 8, 2, 1, 3, 7],
       [7, 2, 8, 1, 3, 9, 4, 5, 6],
       [6, 4, 2, 9, 1, 8, 3, 7, 5],
       [8, 5, 3, 4, 6, 7, 9, 2, 1],
       [9, 7, 1, 2, 5, 3, 6, 8, 4]]

solve_sudoku(sudoku_ultimate)

array([[2, 3, 7, 8, 4, 1, 5, 6, 9],
       [1, 8, 6, 7, 9, 5, 2, 4, 3],
       [5, 9, 4, 3, 2, 6, 7, 1, 8],
       [3, 1, 5, 6, 7, 4, 8, 9, 2],
       [4, 6, 9, 5, 8, 2, 1, 3, 7],
       [7, 2, 8, 1, 3, 9, 4, 5, 6],
       [6, 4, 2, 9, 1, 8, 3, 7, 5],
       [8, 5, 3, 4, 6, 7, 9, 2, 1],
       [9, 7, 1, 2, 5, 3, 6, 8, 4]])

In [ ]:
sudoku_hard = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

solve_sudoku(sudoku_hard)

(0, 2) [8, 1, 3]
(1, 7) [8, 4, 6]
(3, 3) [8, 1]
(0, 7) [8, 3]
(3, 2) [8, 3]


array([[8, 1, 2, 7, 5, 3, 6, 4, 9],
       [9, 4, 3, 6, 8, 2, 1, 7, 5],
       [6, 7, 5, 4, 9, 1, 2, 8, 3],
       [1, 5, 4, 2, 3, 7, 8, 9, 6],
       [3, 6, 9, 8, 4, 5, 7, 2, 1],
       [2, 8, 7, 1, 6, 9, 5, 3, 4],
       [5, 2, 1, 9, 7, 4, 3, 6, 8],
       [4, 3, 8, 5, 2, 6, 9, 1, 7],
       [7, 9, 6, 3, 1, 8, 4, 5, 2]])